In [1]:
%matplotlib inline

# 모델 파라미터 최적화

이제 모델과 데이터가 있으므로 데이터에 대한 파라미터를 최적화하여 모델을 훈련, 검증 및 평가할 차례입니다. 모델 훈련은 반복적인 과정입니다. 각 반복(**epoch**)에서 모델은 출력값을 예측하고, 예측값에서 오류(**loss**)를 계산하고, 파라미터와 관련하여 오류의 도함수를 수집하며, 경사 하강법을 사용하여 이러한 파라미터를 최적화합니다. 이 프로세스에 대한 자세한 설명은 [동영상 링크](https://www.youtube.com/watch?v=tIeHLnjs5U8)를 확인하십시오.


## Prerequisite code 

**Datasets & DataLoaders** 및 **Build Model**을 통해 이전 모듈의 코드를 불러옵니다.


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터 세팅

Hyperparameter는 모델 최적화 프로세스를 제어할 수 있는 조정 가능한 파라미터입니다. 다른 hyperparameter 값은 모델 학습 및 수렴률에 영향을 줄 수 있습니다.
(Hyperparameter Tuning에 대해 자세히 알아보려면 [동영상 링크](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)를 참조하십시오.)

훈련을 위해 다음과 같은 Hyperparameter를 정의합니다.
 - **Number of Epochs** : 총 반복 횟수
 - **Batch Size** : 각 Epoch별 모델에서 사용할 데이터 개수
 - **Learning Rate** : 각 Batch / Epoch에서 모델 매개 변수를 업데이트할 양. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중에 예측할 수 없는 동작이 발생할 수 있습니다.

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Add an optimization loop

hyperparameter를 설정하면 최적화 루프(loop)를 사용하여 모델을 훈련하고 최적화할 수 있습니다. 최적화 루프의 각 반복을 **epoch**라고 합니다.

각각의 epoch는 두 파트로 구성됩니다.
 - **The Train Loop** : 훈련 데이터을 반복하고 최적의 파라미터로 수렴합니다.
 - **The Validation/Test Loop** : 모델 성능이 개선되고 있는지 확인하기 위해 검증 및 테스트 데이터셋을 반복합니다.
훈련 루프에서 사용되는 몇 가지 개념에 대해 간략히 살펴보겠습니다. 최적화 루프의 `full-impl-label`을 보려면 건너뛰십시오.


### Add a loss function

훈련 데이터가 제공되면 훈련되지 않은 네트워크는 정답을 제공하지 않을 가능성이 높습니다. 손실 함수(Loss Function)는 예측값과 실제값의 비유사 정도를 측정하는 것으로, 훈련 중 Loss를 최소화하도록 계산합니다. Loss를 계산하기 위해 주어진 데이터 샘플을 입력하여 예측하고, 이를 실제 값과 비교합니다.

일반적인 손실 함수에는 회귀 작업을 위한 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)(Mean Square Error)와 분류 작업을 위한 [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood)가 있습니다. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)는 `nn.LogSoftmax`와 `nn.NLLLoss`를 결합합니다.

모델의 출력 logits을 `nn.CrossEntropyLoss`에 전달하여 logits을 정규화하고, 예측 오류를 계산합니다.

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

최적화는 각 학습 단계에서 모델 error를 줄이기 위해 모델의 파라미터를 조정하는 프로세스입니다. 최적화 알고리즘은 이 프로세스가 수행되는 방식을 정의하며, 아래 예제에서는 확률적 경사 하강법(Stochastic Gradient Descent; SGD)을 사용합니다.
모든 최적화 로직은 ``optimizer`` 객체에 캡슐화(encapsulated)되며, 여기에서는 SGD 옵티마이저를 사용합니다. 또한 ADAM 및 RMSProp과 같이 PyTorch에는 다양한 종류의 모델 및 데이터에 대해 더 잘 작동하는 최적화 프로그램이 있습니다.

학습해야 하는 모델 파라미터를 등록하고 hyperparameter인 학습률을 설정하여 최적화 프로그램을 초기화합니다.

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

훈련 루프 내에서 최적화는 세 단계로 이루어집니다.
 * 모델 파라미터의 기울기를 재설정하려면 `optimizer.zero_grad()`를 호출합니다. 기본적으로 기울기가 추가되며, 이중 계산을 방지하기 위해 반복할 때마다 명시적으로 0으로 설정해줍니다.
 * `loss.backwards()`를 호출하여 loss를 역전파해줍니다.
 * 기울기가 존재하면 ``optimizer.step()``을 호출하여 역전파 과정에서 수집된 기울기로 파라미터를 조정합니다.

## Full implementation

최적화 코드를 반복하는 `train_loop`와 평가 데이터에 대한 모델 성능을 평가하는 `test_loop`를 정의합니다.

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고 `train_loop`와 `test_loop`에 전달합니다. 모델의 성능 향상을 추적하기 위해 epoch 수를 자유롭게 조절하십시오.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307260  [    0/60000]
loss: 2.305284  [ 6400/60000]
loss: 2.293966  [12800/60000]
loss: 2.291592  [19200/60000]
loss: 2.288022  [25600/60000]
loss: 2.259277  [32000/60000]
loss: 2.277950  [38400/60000]
loss: 2.252569  [44800/60000]
loss: 2.238333  [51200/60000]
loss: 2.239141  [57600/60000]
Test Error: 
 Accuracy: 27.5%, Avg loss: 0.035050 

Epoch 2
-------------------------------
loss: 2.222609  [    0/60000]
loss: 2.244805  [ 6400/60000]
loss: 2.209550  [12800/60000]
loss: 2.227453  [19200/60000]
loss: 2.217051  [25600/60000]
loss: 2.162092  [32000/60000]
loss: 2.206926  [38400/60000]
loss: 2.151579  [44800/60000]
loss: 2.117667  [51200/60000]
loss: 2.143689  [57600/60000]
Test Error: 
 Accuracy: 38.9%, Avg loss: 0.033368 

Epoch 3
-------------------------------
loss: 2.102783  [    0/60000]
loss: 2.154025  [ 6400/60000]
loss: 2.076486  [12800/60000]
loss: 2.124048  [19200/60000]
loss: 2.107713  [25600/60000]
loss: 2.014179  [32000/600

모델이 처음에는 성능이 좋지 않다는 것을 알았을 것입니다. 더 많은 `epochs`에 대해 루프를 실행하거나 `learning_rate`를 더 큰 숫자로 조정해보세요. 또한 우리가 선택한 모델 구성이 이러한 종류의 문제에 대한 최적의 구성이 아닐 수도 있습니다. 이후 과정에서는 Vision 문제에 대해 작동하는 모델 구조에 대해 자세히 알아볼 것입니다.